In [2]:
import pandas as pd
import transformers as ts

## Load cleaned reviews

In [ ]:
df_cleaned = pd.read_csv('../data/processed/cleaned_reviews_final.csv')
print("Columns in cleaned reviews:")
df_cleaned.head()
print("Shape of cleaned reviews:")
df_cleaned.shape

Columns in cleaned reviews:
Shape of cleaned reviews:


(2135, 5)

In [5]:
print(df_cleaned.shape)

(2135, 5)


In [6]:
## undersAD WHT THE BANK DISTRIBUTION LOOK LIKE
print(df_cleaned['bank'].value_counts())

bank
BOA       780
CBE       747
Dashen    608
Name: count, dtype: int64


In [7]:
##check missing text
print(f"Number of missing reviews (NaN): {df_cleaned['review'].isnull().sum()}")
print(f"Number of empty reviews: {(df_cleaned['review'] == '').sum()}")
print(f"Total missing or empty reviews: {(df_cleaned['review'].isnull() | (df_cleaned['review'] == '')).sum()}")

Number of missing reviews (NaN): 0
Number of empty reviews: 0
Total missing or empty reviews: 0


In [9]:
## check number of duplicated rows
print(f"Number of duplicate rows: {df_cleaned.duplicated().sum()}")

Number of duplicate rows: 0


##Use distilbert-base-uncased-finetuned-sst-2-english model.

In [11]:
##Load pipeline
from transformers import pipeline

sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

Device set to use cpu


In [14]:
## test the sentiment analyzer

example_reviews = [
    "This app is amazing and works perfectly! I love it.",
    "the app is too slow and not user-friendly"
]

sentiment_results = sentiment_analyzer(example_reviews)

for review, result in zip(example_reviews, sentiment_results):
    print(f"Review: '{review}'")
    print(f"Sentiment: {result['label']} (Score: {result['score']:.2f})\n")

Review: 'This app is amazing and works perfectly! I love it.'
Sentiment: POSITIVE (Score: 1.00)

Review: 'the app is too slow and not user-friendly'
Sentiment: NEGATIVE (Score: 1.00)



In [17]:
## loop throw the reviws and return the sentiment label and score in new column
def analyze_sentiment(review_text):
    # Ensure review_text is a string, as some entries might be non-string (e.g., NaN, though checked earlier)
    if not isinstance(review_text, str):
        return {'label': 'UNKNOWN', 'score': 0.0} # Handle non-string entries
    # The pipeline returns a list of dictionaries, we want the first (and only) element
    return sentiment_analyzer(review_text)[0]

# Apply the function to the 'review' column and store the results in a new column
df_cleaned['sentiment_results'] = df_cleaned['review'].apply(analyze_sentiment)

# Extract 'label' and 'score' into separate columns
df_cleaned['sentiment_label'] = df_cleaned['sentiment_results'].apply(lambda x: x['label'])
df_cleaned['sentiment_score'] = df_cleaned['sentiment_results'].apply(lambda x: x['score'])

# Remove the intermediate 'sentiment_results' column
df_cleaned = df_cleaned.drop(columns=['sentiment_results'])

# Display the first few rows with the new sentiment label and score columns
display(df_cleaned.head())


,review,rating,date,bank,source,sentiment_label,sentiment_score
0,this application is very important and advanta...,5,2025-11-27,CBE,Google Play Store,POSITIVE,0.998468
1,why didn't work this app?,1,2025-11-27,CBE,Google Play Store,NEGATIVE,0.999132
2,the app makes our life easier. thank you cbe!,5,2025-11-27,CBE,Google Play Store,POSITIVE,0.999696
3,this app very bad 👎,1,2025-11-27,CBE,Google Play Store,NEGATIVE,0.999797
4,the most advanced app. but how to stay safe?,5,2025-11-27,CBE,Google Play Store,NEGATIVE,0.956509


In [ ]:
##"What is the distribution of sentiment per bank?"

In [18]:
sentiment_by_bank = df_cleaned.groupby('bank').agg(
    total_reviews=('review', 'count'),
    positive_count=('sentiment_label', lambda x: (x == 'POSITIVE').sum()),
    negative_count=('sentiment_label', lambda x: (x == 'NEGATIVE').sum()),
    average_sentiment_score=('sentiment_score', 'mean')
)

sentiment_by_bank['positive_percentage'] = (sentiment_by_bank['positive_count'] / sentiment_by_bank['total_reviews']) * 100
sentiment_by_bank['negative_percentage'] = (sentiment_by_bank['negative_count'] / sentiment_by_bank['total_reviews']) * 100

display(sentiment_by_bank)

,total_reviews,positive_count,negative_count,average_sentiment_score,positive_percentage,negative_percentage
bank,,,,,,
BOA,780,221,559,0.964558,28.333333,71.666667
CBE,747,408,339,0.962613,54.618474,45.381526
Dashen,608,394,214,0.979894,64.802632,35.197368


In [19]:
df_cleaned.to_csv('../data/processed/reviews_with_sentiment.csv', index=False)
print("DataFrame with sentiment results saved to 'reviews_with_sentiment.csv'")

DataFrame with sentiment results saved to 'reviews_with_sentiment.csv'


## thematic_analysis for the projrect 

In [ ]:
import pandas as pd
import spacy
from collections import defaultdict

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Load cleaned reviews
df = pd.read_csv("../data/processed/cleaned_reviews.csv")

print("Loaded:", df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/cleaned_reviews.csv'